In [ ]:
# refrence :
# https://www.codementor.io/jadianes/data-science-python-pandas-r-dimensionality-reduction-du1081aka

In [1]:
import numpy as np
import pandas as pd
import _pickle as pickle
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv('./data/data_clean0.csv.zip', compression='zip')
#data = pd.read_csv('./data/data_clean0.csv')

In [3]:
# present some info
print(data.info())
print(data.shape)
data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53801 entries, 0 to 53800
Columns: 4991 entries, 0 to 4990
dtypes: float64(4991)
memory usage: 2.0 GB
None
(53801, 4991)


,0,1,2,3,4,5,6,7,8,9,...,4981,4982,4983,4984,4985,4986,4987,4988,4989,4990
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006403,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005821,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# get correlations and keep track of Highly correlated columns for later use.
flds = data.columns
cols = len(flds)
#corr = data.iloc[:4459].corr().values
corr = np.load('./data/all_correlations.npy')
imp_corr = []
track_cols = set()

for i in range(cols):
    for j in range(i+1, cols):
        # if both items already written, don't bother to rewrite them again
        if i in track_cols and j in track_cols :
            continue
        if corr[i,j] >= .7 or corr[i,j] <= -.7 :
            imp_corr.append((flds[i], flds[j], corr[i,j]))
            track_cols.add(i)
            track_cols.add(j)
            #print("{:4d}  {:4d}  {:5f}".format(flds[i],flds[j], corr[i,j]))

#np.save('./data/all_correlations.npy', corr)
with open('./data/imp_correlations.pkl', 'wb') as f:
    pickle.dump(imp_corr, f)

print('there are %d important correlations with %d total columns'%(len(imp_corr),len(track_cols))) 

there are 1049 important correlations with 1569 total columns


In [5]:
# now group imp correlations into lists so it's easier to use them
grouped_high_corr = []
holder = []
s = -100
for i in imp_corr:
    if s != i[0]:
        if len(holder) > 0 :
            grouped_high_corr.append(holder)
            holder = []
        s = i[0]
        holder.append(s)
    
    holder.append(i[1])
#-----------------------------
print('%d groups'%len(grouped_high_corr))

653 groups


In [6]:
# Instead to drop duplicates, I'll use PCA to reduce each group to one or two columns 
#  without losing information.
PCs_data = np.empty([data.shape[0], 0])
cou = 0
for g in grouped_high_corr:
    group_data = data[g].copy()
    # make sure the data in centered around 0 mean.
    #group_data -= np.mean(group_data, axis=0)
    #scaler = MinMaxScaler()
    #group_data = scaler.fit_transform(group_data)
    # define PCA classifier
    pca = PCA(n_components=2)
    group_PC = pca.fit_transform(group_data)
    # if second variance_ratio isn't big enough, I'll just drop it.
    if pca.explained_variance_ratio_[1] < 0.5 :
        group_PC = group_PC[:,0].reshape((-1,1))
        cou += 1
    # now add the new data to the large dataframe
    PCs_data = np.concatenate((PCs_data, group_PC), axis=1)
#---------------------------------------------
print('we reduced %d columns to %d columns'%(len(track_cols),PCs_data.shape[1]) )

we reduced 1569 columns to 653 columns


In [7]:
# convert the array into dataframe, set names to the columns
columns = ['pc_corr_%d' % i for i in range(PCs_data.shape[1])]
PCs_df = pd.DataFrame(data=PCs_data, columns=columns)
# drop the columns we used in PCA and then add the 2 dfs together
# I need first to convert track_cols elements from int to str.
cols_to_drop = ['%d' % i for i in track_cols]
data.drop(columns=cols_to_drop, inplace=True)
data = pd.concat((data, PCs_df), axis=1)
# present some info
data.head(5)
data.info()
data.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53801 entries, 0 to 53800
Columns: 4075 entries, 1 to pc_corr_652
dtypes: float64(4075)
memory usage: 1.6 GB


(53801, 4075)

In [8]:
# perfect we finished :D
# save to disk
data.to_csv('./data/data_clean1.csv.zip',compression='zip', index=False)

In [ ]:
import sys

sl = []
i = 0
# some magic 1024 - overhead of string object
fill_size = 1024
if sys.version.startswith('2.7'):
    fill_size = 1003
if sys.version.startswith('3'):
    fill_size = 497
print(fill_size)
MiB = 0
while True:
    s = str(i).zfill(fill_size)
    sl.append(s)
    if i == 0:
        try:
            sys.stderr.write('size of one string %d\n' % (sys.getsizeof(s)))
        except AttributeError:
            pass
    i += 1
    if i % 1024 == 0:
        MiB += 1
        if MiB % 25 == 0:
            sys.stderr.write('%d [MiB]\n' % (MiB))